In [13]:
import requests
import pandas as pd

def get_listing(page):
    cookies = {
        '_vwo_uuid_v2': 'D0D4CA9A0BCC6DB27F80D25A763C4D6CE|7b5fd0a966bda1b5ce93a6ee7de8d423',
        'huntr_referrer': '',
        '_vwo_uuid': 'D0D4CA9A0BCC6DB27F80D25A763C4D6CE',
        '_vwo_ds': '3%241745493630%3A62.233805%3A%3A',
        '_vis_opt_s': '1%7C',
        '_vis_opt_test_cookie': '1',
        '_gid': 'GA1.2.562523841.1745493631',
        '_rdt_uuid': '1745493630646.9e0be9ac-722d-4dde-b2c2-94fcac55223a',
        'mp_1d6f823f8d5432354b55fc0af879343c_mixpanel': '%7B%22distinct_id%22%3A%22%24device%3Adbf2052c-4643-40ad-9396-804ea81e6f96%22%2C%22%24device_id%22%3A%22dbf2052c-4643-40ad-9396-804ea81e6f96%22%2C%22%24initial_referrer%22%3A%22%24direct%22%2C%22%24initial_referring_domain%22%3A%22%24direct%22%7D',
        '_vwo_sn': '0%3A2%3A%3A%3A1',
        '_ga_JYDVJK9187': 'GS1.1.1745493631.1.1.1745493660.31.0.298614848',
        '_ga': 'GA1.1.2072904394.1745493631',
        '_cioanonid': '6c068de4-ed09-59d1-3ea7-62b8fcb243fe',
        '__client_uat': '0',
        '__client_uat_NHkmJr6b': '0',
        'ph_phc_UCmuRvZQZKMwWzoDGtP9nOTXBBEJ7kq7jUmR6zKAbCG_posthog': '%7B%22distinct_id%22%3A%2201966787-1b56-7c16-b6c9-f9ab063b8364%22%2C%22%24sesid%22%3A%5B1745493733690%5D%7D',
    }

    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'priority': 'u=1, i',
        'referer': 'https://huntr.co/jobs',
        'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
    }

    params = {
        'page': str(page),
        'zoom': '1',
    }

    try:
        response = requests.get('https://huntr.co/public/search/job-posts', params=params, cookies=cookies, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        try:
            json_response = response.json()  # Parse the JSON response
            return json_response
        except ValueError as e:  # Catch JSON decoding errors
            print(f"JSON decode error: {e}")
            return None
    except requests.exceptions.RequestException as e:  # Catch other requests-related errors
        print(f"HTTP request error: {e}")
        return None

In [7]:

page = 1  
json_response = get_listing(page)

if json_response and 'results' in json_response:
    master_list = []
    for listing in json_response['results']:
        data_dict = {}
        data_dict['id'] = listing.get('_id', '')
        data_dict['created_at'] = listing.get('createdAt', '')
        data_dict['multi_city'] = listing.get('isMultiCity', '')
        data_dict['remote'] = listing.get('isRemote', '')
        data_dict['last_crawl_date'] = listing.get('latestCrawlDate', '')
        data_dict['original_job_category'] = listing.get('originalJobCategory', '')
        data_dict['original_location'] = listing.get('originalLocations', [])[0] if listing.get('originalLocations') else 'N/A'
        data_dict['original_post_date'] = listing.get('originalPostDate', '')
        data_dict['original_title'] = listing.get('originalTitle', '')

        try:
            data_dict['place_name'] = listing['places'][0]['name']
        except:
            data_dict['place_name'] = 'N/A'

        try:
            data_dict['place_address'] = listing['places'][0]['address']
        except:
            data_dict['place_address'] = 'N/A'

        try:
            data_dict['place_country'] = listing['places'][0]['country']
        except:
            data_dict['place_country'] = 'N/A'

        try:
            data_dict['place_lon'] = listing['places'][0]['point']['coordinates'][0]
            data_dict['place_lat'] = listing['places'][0]['point']['coordinates'][1]
        except:
            data_dict['place_lon'] = 'N/A'
            data_dict['place_lat'] = 'N/A'

        data_dict['textDescription'] = listing.get('textDescription', '')
        data_dict['url'] = listing.get('url', '')

        try:
            data_dict['employer_name'] = listing['employer']['name']
        except:
            data_dict['employer_name'] = 'N/A'

        try:
            data_dict['employer_legalName'] = listing['employer']['legalName']
        except:
            data_dict['employer_legalName'] = 'N/A'

        try:
            data_dict['employer_domain'] = listing['employer']['domain']
        except:
            data_dict['employer_domain'] = 'N/A'

        try:
            data_dict['employer_employees'] = listing['employer']['employees']
        except:
            data_dict['employer_employees'] = 'N/A'

        try:
            data_dict['employer_employeeRange'] = listing['employer']['employeeRange']
        except:
            data_dict['employer_employeeRange'] = 'N/A'

        # data_dict['tags'] = listing.get('tags', 'N/A')
        # data_dict['roles'] = listing.get('roles', 'N/A')

        master_list.append(data_dict)
else:
    print("No results found in the JSON response.")

In [8]:
listing.keys()

dict_keys(['originalLocations', 'isRemote', 'isMultiCity', 'isPrivate', 'createdByApi', 'jobPostVisibility', 'sourceName', '_roles', '_offices', '_skills', '_tags', '_organizations', '_id', '__v', '_employer', 'createdAt', 'employeeCount', 'htmlDescription', 'latestCrawlDate', 'originalJobCategory', 'originalJobType', 'originalPostDate', 'originalTitle', 'places', 'postDate', 'textDescription', 'updatedAt', 'url', 'firstCrawlDate', 'company', 'organizationCompany', 'employer', 'tags', 'roles', 'salary', 'place', 'id'])

In [14]:
master_list[0]

{'id': '6809a096a235e60054d3236b',
 'created_at': '2025-04-24T02:23:18.454Z',
 'multi_city': False,
 'remote': False,
 'last_crawl_date': '2025-04-24T02:23:17.601Z',
 'original_job_category': 'Sales',
 'original_location': 'Atlanta Georgia United States',
 'original_post_date': '2025-4-23',
 'original_title': 'B2B Sales Account Executive - Fiber',
 'place_name': 'Atlanta, Fulton County, Georgia, USA',
 'place_address': 'Atlanta, Fulton County, Georgia, USA',
 'place_country': 'United States of America',
 'place_lon': -84.3902644,
 'place_lat': 33.7489924,
 'textDescription': "Job Description:Sales employees will concentrate on acquiring new business and conducting customer visits in the field. When not in the field, they are expected to collaborate with their team from their assigned office location, if available.Joining our team comes with perks! The AT&T Fiber team is expanding their sales team and hiring fast! Join the team and receive a $3,500 sign on bonus, Don’t delay, review the

In [12]:
df = pd.DataFrame(master_list)
df

,id,created_at,multi_city,remote,last_crawl_date,original_job_category,original_location,original_post_date,original_title,place_name,...,place_country,place_lon,place_lat,textDescription,url,employer_name,employer_legalName,employer_domain,employer_employees,employer_employeeRange
0,6809a096a235e60054d3236b,2025-04-24T02:23:18.454Z,False,False,2025-04-24T02:23:17.601Z,Sales,Atlanta Georgia United States,2025-4-23,B2B Sales Account Executive - Fiber,"Atlanta, Fulton County, Georgia, USA",...,United States of America,-84.390264,33.748992,Job Description:Sales employees will concentra...,https://www.att.jobs/job/atlanta/b2b-sales-acc...,AT&T,AT&T Inc.,att.com,18749,1000+
1,6809a0968e2bd600492b5880,2025-04-24T02:23:18.438Z,False,False,2025-04-24T02:23:17.987Z,Sales,Garland Texas United States,2025-4-23,Bilingual Spanish Retail Sales Consultant Garland,"Garland, Dallas County, Texas, USA",...,United States of America,-96.638883,32.912624,Job Description:Do you speak Spanish and Engli...,https://www.att.jobs/job/garland/bilingual-spa...,AT&T,AT&T Inc.,att.com,18749,1000+
2,6809a0958e2bd600492b5838,2025-04-24T02:23:17.304Z,False,False,2025-04-24T02:23:16.925Z,Sales,Chicago Illinois United States,2025-4-23,Retail Sales Consultant - WEST LOOP,"Chicago, Cook County, Illinois, USA",...,United States of America,-87.624421,41.875562,Job Description:JobTitle : Retail Sales Consul...,https://www.att.jobs/job/chicago/retail-sales-...,AT&T,AT&T Inc.,att.com,18749,1000+
3,6809a095caf504005055e40c,2025-04-24T02:23:17.062Z,False,False,2025-04-24T02:23:16.512Z,,Columbia Maryland United States,2025-4-23,End-User System Engineer (SE II) (Government),"Columbia, Howard County, Maryland, USA",...,United States of America,-76.858205,39.215621,Job Description:AT&T Global Public Sector is a...,https://www.att.jobs/job/columbia/end-user-sys...,AT&T,AT&T Inc.,att.com,18749,1000+
4,6809a095a235e60054d322ed,2025-04-24T02:23:17.033Z,False,False,2025-04-24T02:23:16.636Z,Sales,Elk Grove California United States,2025-4-23,Bilingual Spanish Retail Sales Consultant ELK ...,"Elk Grove, Sacramento County, California, USA",...,United States of America,-121.371618,38.408799,Job Description:Do you speak Spanish and Engli...,https://www.att.jobs/job/elk-grove/bilingual-s...,AT&T,AT&T Inc.,att.com,18749,1000+
5,6809a094caf504005055e367,2025-04-24T02:23:16.355Z,False,False,2025-04-24T02:23:16.089Z,,Charlotte North Carolina United States,2025-4-23,Lead Cybersecurity VPN Security Engineer,"Charlotte, Mecklenburg County, North Carolina,...",...,United States of America,-80.843083,35.227209,Job Description:Join AT&T and reimagine the co...,https://www.att.jobs/job/charlotte/lead-cybers...,AT&T,AT&T Inc.,att.com,18749,1000+
6,6809a0931850d4004f7673b2,2025-04-24T02:23:15.163Z,False,False,2025-04-24T02:23:14.970Z,,Dallas Texas United States,2025-4-23,Lead Product Marketing,"Dallas, Dallas County, Texas, USA",...,United States of America,-96.796856,32.776272,Job Description:Location: This position requir...,https://www.att.jobs/job/dallas/lead-product-m...,AT&T,AT&T Inc.,att.com,18749,1000+
7,6809a0911850d4004f76730d,2025-04-24T02:23:13.763Z,False,False,2025-04-24T02:23:13.533Z,,Middletown New Jersey United States,2025-4-23,Lead Cybersecurity VPN Security Engineer,"Middletown, Orchard Street, Middletown Townshi...",...,United States of America,-74.116215,40.389819,Job Description:Join AT&T and reimagine the co...,https://www.att.jobs/job/middletown/lead-cyber...,AT&T,AT&T Inc.,att.com,18749,1000+
8,6809a090a235e60054d32246,2025-04-24T02:23:12.360Z,False,False,2025-04-24T02:23:11.865Z,,Bothell Washington United States,2025-4-23,Lead Cybersecurity VPN Security Engineer,"Bothell, King County, Washington, 98011, USA",...,United States of America,-122.205404,47.762320,Job Description:Join AT&T and reimagine the co...,https://www.att.jobs/job/bothell/lead-cybersec...,AT&T,AT&T Inc.,att.com,18749,1000+
9,6809a090caf504005055e2aa,2025-04-24T02:23:12.287Z,False,False,2025-04-24T02:23:11.8

In [16]:
df.to_csv('page_1.csv', index=False)